<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Практическая работа № 4  
**Тема.** Классификация текста с помощью глубокого обучения и трансформерных моделей

---

## 1. Цель и задачи работы

**Цель работы** — сформировать у обучающегося целостное, системное и практическое понимание современного ландшафта архитектур глубокого обучения, применяемых для решения задач текстовой классификации. Особое внимание уделяется не только реализации моделей, но и **глубокому пониманию их внутреннего устройства**: от простейших многослойных персептронов до сложных трансформерных систем с механизмами внимания. Работа призвана продемонстрировать, как эволюция архитектур — от рекуррентных сетей с энкодер-декодерной структурой до современных трансформеров — позволила решить фундаментальные ограничения ранних подходов, такие как проблема долгосрочной зависимости и неэффективность последовательной обработки.

**Основные задачи работы:**

1. Использовать единый корпус текстов, собранный в Практической работе № 1 и размеченный в Практической работе № 3, в качестве общей экспериментальной платформы для всех моделей.  
2. Реализовать и экспериментально сравнить **полный спектр архитектур глубокого обучения**:  
   - **Базовые модели**: многослойный персептрон (MLP), свёрточные нейронные сети (CNN), простые рекуррентные сети (RNN);  
   - **Расширенные рекуррентные архитектуры**: LSTM, GRU, двунаправленные (BiLSTM, BiGRU), стекированные (Stacked RNN), а также **энкодер-декодерные схемы** с и без механизма внимания;  
   - **Трансформерные модели**: BERT, GPT, а также специализированные русскоязычные версии — **RuBERT** (DeepPavlov), **ruRoBERTa** (Sber AI), **DistilRuBERT**, **rubert-tiny**, **mDeBERTa-v3-base** и другие актуальные модели.  
3. Исследовать **роль механизма внимания** — от его первоначального применения в энкодер-декодерных системах до полной замены рекуррентности в архитектуре трансформера.  
4. Обеспечить **комплексную визуализацию** процесса обучения и результатов: графики потерь и метрик по эпохам, матрицы ошибок, ROC- и PR-кривые, 2D-проекции эмбеддингов, тепловые карты внимания.  
5. Применить современные методы борьбы с дисбалансом классов в условиях глубокого обучения.  
6. Проанализировать **интерпретируемость предсказаний** с использованием градиентных методов (saliency maps), визуализации весов внимания и библиотек типа **Captum**.  
7. Разработать интерактивный веб-интерфейс, позволяющий сравнивать все реализованные модели на единой платформе.  
8. Обеспечить воспроизводимость через публикацию кода, fine-tuned моделей и интерфейса.

---

## 2. Теоретические предпосылки

Текстовая классификация с помощью глубокого обучения прошла несколько этапов эволюции, каждый из которых решал определённые ограничения предыдущего поколения.

### 2.1. Базовые архитектуры: MLP и CNN

- **Многослойный персептрон (MLP)** представляет собой полносвязную нейронную сеть, принимающую на вход фиксированный вектор признаков (например, усреднённый эмбеддинг документа). Несмотря на простоту, MLP служит важным базовым эталоном, позволяющим оценить, насколько сложные архитектуры действительно превосходят «наивный» подход.  
- **Свёрточные нейронные сети (CNN)** были адаптированы для текста из компьютерного зрения. Используя 1D-свёртки с различными размерами ядер (например, 3, 4, 5), CNN эффективно выявляют локальные семантические паттерны — своего рода «ключевые n-граммы», определяющие тему или тональность. Архитектура Kim (2014) остаётся классическим примером применения CNN в NLP.

### 2.2. Рекуррентные сети и их эволюция

Простые RNN страдают от **проблемы исчезающих и взрывающихся градиентов**, что делает их неспособными моделировать долгосрочные зависимости. Эта проблема была решена с появлением:

- **LSTM (Long Short-Term Memory)** и **GRU (Gated Recurrent Unit)** — модифицированных RNN с вентильными механизмами, контролирующими поток информации через ячейку памяти.  
- **Двунаправленные архитектуры (BiRNN)**, в которых текст обрабатывается в прямом и обратном направлениях, что позволяет каждому токену «видеть» как прошлый, так и будущий контекст.  
- **Стекированные RNN (Stacked RNN)**, где выход одного слоя RNN подаётся на вход следующего, что позволяет извлекать иерархические признаки на разных уровнях абстракции.

Критическим прорывом стало введение **энкодер-декодерной архитектуры** (Sutskever et al., 2014), первоначально разработанной для машинного перевода. В этой схеме:
- **Энкодер** (обычно LSTM или GRU) преобразует входную последовательность в фиксированный **контекстный вектор**;
- **Декодер** генерирует выходную последовательность на основе этого вектора.

Однако фиксированный контекстный вектор оказывался «узким местом» для длинных текстов. Это привело к появлению **механизма внимания (attention)** (Bahdanau et al., 2015), который позволил декодеру **динамически фокусироваться на разных частях входной последовательности** на каждом шаге генерации. В задачах классификации декодер часто заменяется простым классификационным слоем, но механизм внимания сохраняется для выявления наиболее релевантных слов.

### 2.3. Эра трансформеров

Архитектура **трансформера** (Vaswani et al., 2017) полностью отказалась от рекуррентности, заменив её **механизмом само-внимания (self-attention)**, который позволяет каждому токену напрямую взаимодействовать со всеми другими токенами в последовательности. Это обеспечивает:
- параллельную обработку (ускорение обучения);
- эффективное моделирование как локальных, так и глобальных зависимостей.

Для русского языка наибольшее распространение получили:
- **RuBERT** (DeepPavlov) — BERT, дообученный на Национальном корпусе русского языка и новостях;
- **ruRoBERTa** (Sber AI) — более мощная версия, обученная на 180 ГБ текстов с динамической маскировкой;
- **DistilRuBERT**, **rubert-tiny** — компактные версии, полученные методами дистилляции знаний, подходящие для развёртывания на устройствах с ограниченными ресурсами;
- **mDeBERTa-v3-base** — мультиязычная модель с улучшенным представлением позиций и маски внимания.

Все эти модели используют процедуру **fine-tuning**: предобученная модель адаптируется под конкретную задачу классификации путём дообучения небольшого числа слоёв (часто — только головного классификационного слоя).

---

## 3. Методика и порядок выполнения работы

### 3.1. Этап 1. Подготовка экспериментального корпуса

Используется **единый корпус** из Практической работы № 1, дополненный разметкой из Работы № 3:
- **Бинарная классификация**: тональность (позитив/негатив);
- **Многоклассовая**: темы (политика, экономика, спорт, культура);
- **Многометочная**: несколько тем на документ.

**Требования**:  
- Объём — не менее 10 000 документов на тип задачи;  
- Формат — JSONL с полями `text`, `sentiment`, `category`, `categories`;  
- Разделение — 70/15/15 с сохранением стратификации.  
> Важно: для трансформеров текст **не подвергается лемматизации или стеммингу**, чтобы сохранить оригинальную орфографию.

### 3.2. Этап 2. Подготовка данных

- Для **MLP/CNN/RNN**:  
  - Токенизация с использованием ваших подсловных моделей (BPE/WordPiece из Работы № 1) или `spaCy`;  
  - Преобразование в последовательность ID;  
  - Padding/truncation до фиксированной длины (рекомендуется 256 токенов).

- Для **трансформеров**:  
  - Использование официального токенизатора через `AutoTokenizer.from_pretrained`;  
  - Формирование входных тензоров: `input_ids`, `attention_mask`, `token_type_ids` (если требуется).

### 3.3. Этап 3. Реализация моделей глубокого обучения

Создаётся модуль `deep_classifiers.py` с поддержкой:

#### Базовые модели:
- **MLP**: 2–3 полносвязных слоя поверх усреднённого эмбеддинга;
- **CNN**: 1D-свёртки с ядрами 3, 4, 5 → max-pooling → dropout → классификация.

#### Рекуррентные модели:
- **Simple RNN**, **LSTM**, **GRU**;
- **BiLSTM**, **BiGRU**;
- **Stacked LSTM/GRU** (2–3 слоя);
- **Энкодер-декодер с attention**:  
  - Энкодер: BiLSTM;  
  - Механизм внимания: вычисление весов на основе скрытых состояний энкодера и текущего состояния декодера;  
  - Декодер (для классификации заменяется на слой внимания + классификатор).

#### Трансформерные модели:
- `bert-base-multilingual-cased`;  
- `DeepPavlov/rubert-base-cased`;  
- `sberbank-ai/ruRoberta-large`;  
- `cointegrated/rubert-tiny2`;  
- `microsoft/mdeberta-v3-base`.

Для всех трансформеров добавляется линейный классификационный слой поверх выхода `[CLS]`-токена.

### 3.4. Этап 4. Обучение и комплексная визуализация

На этом этапе акцент делается не только на метриках, но и на **наглядном представлении процесса и результатов**:

- **Графики обучения**: loss и F1 по эпохам для train/validation (для выявления переобучения);  
- **Матрицы ошибок (confusion matrices)**: для всех типов задач (нормализованные и абсолютные);  
- **ROC- и PR-кривые**: для бинарной и многоклассовой (one-vs-rest);  
- **2D-проекции**: UMAP/t-SNE на векторах `[CLS]` или усреднённых эмбеддингах с цветовой разметкой классов;  
- **Тепловые карты внимания**: для трансформеров — визуализация весов self-attention между токенами.

### 3.5. Этап 5. Борьба с дисбалансом классов

- Взвешенная кросс-энтропия (`weight` в PyTorch);  
- **Focal Loss** — снижает вклад легко классифицируемых примеров;  
- Использование **F1** и **PR-AUC** как основных метрик при дисбалансе > 3:1.

### 3.6. Этап 6. Интерпретация предсказаний

- Для **трансформеров**:  
  - Визуализация attention heads (например, через `bertviz`);  
  - **Saliency maps** — вычисление градиента выхода по входным эмбеддингам (Captum);  
- Для **RNN/CNN**:  
  - Анализ активаций последнего слоя;  
  - Для CNN — выделение наиболее активных фильтров и соответствующих n-грамм.

### 3.7. Этап 7. Разработка веб-интерфейса

Интерфейс (рекомендуется **Gradio**) должен позволять:
- ввести текст;
- выбрать тип задачи и модель;
- увидеть:  
  - предсказание и вероятности,  
  - график обучения (для выбранной модели),  
  - матрицу ошибок (на тестовой выборке),  
  - карту внимания или saliency map,  
  - 2D-проекцию положения текста в пространстве эмбеддингов.

### 3.8. Этап 8. Публикация результатов

- Fine-tuned трансформеры — на **Hugging Face Hub** с полным **Model Card**;  
- Код — на GitHub/GitLab с `environment.yml`, инструкцией и примерами;  
- Веб-приложение — развернуто через **Hugging Face Spaces**.

---

## 4. Дополнительные исследовательские задания

1. **Эволюция внимания**: Постройте сравнение: энкодер-декодер без внимания → с вниманием → трансформер. Как меняется качество и интерпретируемость?  
2. **Глубина vs ширина**: Сравните стекированные LSTM (3 слоя) и широкий MLP (1024 нейрона) — что эффективнее для вашей задачи?  
3. **Трансфер от мультиязычных моделей**: Оцените, насколько ruRoBERTa превосходит multilingual BERT и mDeBERTa на русском корпусе.  
4. **Анализ ошибок с вниманием**: Найдите случаи, когда модель ошибается, но карта внимания «смотрит» не туда — можно ли это исправить дообучением?  
5. **Сравнение скорости**: Измерьте время инференса на CPU для rubert-base, rubert-tiny и BiLSTM — при каком объёме данных выгода от качества трансформера окупает его стоимость?

---

## 5. Требования к отчёту

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен включать:

1. Титульный лист;  
2. Введение с обоснованием эволюции архитектур;  
3. Подробное описание всех реализованных моделей, включая схемы энкодер-декодера и механизмов внимания;  
4. Характеристики корпуса;  
5. **Все виды визуализаций** с пояснениями;  
6. Сравнение с классическими методами (Работа № 3);  
7. Примеры интерпретации;  
8. Ссылки на репозиторий, веб-приложение, Hugging Face;  
9. Выводы:  
   - какая архитектура оптимальна для каждой задачи;  
   - какова роль внимания в повышении качества и интерпретируемости;  
   - есть ли практический смысл использовать трансформеры при ограниченных ресурсах.

Список источников — по **ГОСТ Р 7.0.5–2008**.

---

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|----------|
| **Отлично (5)** | Реализованы все типы моделей, включая энкодер-декодер с вниманием и все трансформеры. Есть все виды визуализаций, глубокий анализ внимания, сравнение с классикой. Отчёт — развёрнутый, академичный, с рефлексией. |
| **Хорошо (4)** | Реализованы CNN, LSTM/GRU (включая Bi и Stacked), трансформеры. Есть графики, матрицы, внимание, интерфейс. |
| **Удовлетворительно (3)** | Реализованы хотя бы три архитектуры (например, CNN, BiLSTM, RuBERT). Есть метрики и базовые визуализации. |
| **Неудовлетворительно (2)** | Отсутствует внимание, энкодер-декодер или трансформеры. Нет визуализаций или единого корпуса. |
